In [1]:
import numpy as np
import scipy as sp

import scipy
from scipy.stats import multivariate_normal
from scipy       import optimize

import skimage

import matplotlib.pyplot as plt

import pylab

from glob import glob
import sys
import os

from tqdm import tqdm

import math

from glob import glob

import matplotlib.pyplot as plt

import itk
from itk import TubeTK as ttk

import av
import ffmpeg

import json

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_IO import *
from ARGUSUtils_Linearization import *

In [3]:
def extract_columns(pathname, output_pathname, min_size=16):
    data_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.mha"))
    data_overlay_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.extruded-overlay.mha"))
    print(len(data_paths),len(data_overlay_paths))
    img = itk.GetArrayFromImage(itk.imread(data_paths[0], itk.F))
    print(img.shape)
    num_overlays = len(data_overlay_paths)
    for i in range(num_overlays):
        img = itk.GetArrayFromImage(itk.imread(data_paths[i], itk.F))
        ovrimg = itk.GetArrayFromImage(itk.imread(data_overlay_paths[i], itk.UC))
        print("Processing", data_overlay_paths[i])
        
        # Eliminate ribs and map sliding to 2
        ovrimg = np.where(ovrimg==2,0,ovrimg)
        ovrimg = np.where(ovrimg==3,2,ovrimg)

        nos = 0
        sli = 0
        c = 0
        while c<ovrimg.shape[2]:
            nos = np.count_nonzero(ovrimg[:,:,c]==1)
            rib = 0 #np.count_nonzero(ovrimg[:,:,c]==2)
            sli = np.count_nonzero(ovrimg[:,:,c]==2)
            start_c = c
            end_c = c
            val_c = 0
            name_c = ''
            if nos>rib and sli==0:
                val_c = 1
                name_c = "N"
            #elif rib>nos and rib>sli:
                #val_c = 2
                #name_c = "R"
            elif sli>rib and nos==0:
                val_c = 2  #3
                name_c = "S"
            else:
                c += 1
                continue
            start_c = c
            end_c = c
            c += 1
            count = 0
            if c<ovrimg.shape[2]:
                count = np.count_nonzero(ovrimg[:,:,c]==val_c)
                while count>0 and c<ovrimg.shape[2]-1:
                    c += 1
                    count = np.count_nonzero(ovrimg[:,:,c]==val_c)
            end_c = c
            if end_c-start_c>=min_size:
                y_start = ovrimg.shape[1]
                y_end = 0
                for c in range(start_c,end_c):
                    for z in range(ovrimg.shape[0]):
                        pos = np.argwhere(ovrimg[z,:,c]==val_c)
                        if len(pos)>0:
                            y_start = min(y_start,pos[0][0])
                            y_end = max(y_end,pos[-1][0])
                inshape = list(img.shape)
                outshape = list(img.shape)
                outshape[1] = 224
                outshape[2] = 128
                print(y_start, y_end)
                
                y_start = max(0, y_start - (outshape[1]//5))
                y_end = min(inshape[1], y_start+outshape[1])
                if y_end-y_start<outshape[1]:
                    y_start = y_end-outshape[1]
                
                start_c = max(0, (start_c+end_c)//2 - (outshape[2]//2))
                end_c = min(inshape[2], start_c+outshape[2])
                if end_c-start_c<outshape[2]:
                    start_c = end_c-outshape[2]
                    
                outimg = img[:,y_start:y_end,start_c:end_c]
                outovr = ovrimg[:,y_start:y_end,start_c:end_c]
                outovr2 = np.max(outovr,axis=0)
                #count = np.count_nonzero(outovr!=0,axis=0)
                #outovr2 = np.where(count>outimg.shape[0]//4,val_c,0)
                outovr = np.repeat(outovr2[np.newaxis,:,:], outimg.shape[0], axis=0)

                (filepath,filename) = os.path.split(data_overlay_paths[i])
                
                new_name = output_pathname+"/"+filename.split('.')[0]+'_Class'+name_c+'_'+str(start_c)+'-'+str(end_c)+'.roi.nii.gz'
                itkimg = itk.GetImageFromArray(outimg.astype(np.short))
                itk.imwrite(itkimg, new_name, compression=True)
                
                new_name = output_pathname+"/"+filename.split('.')[0]+'_Class'+name_c+'_'+str(start_c)+'-'+str(end_c)+'.roi.overlay.nii.gz'
                itkimg = itk.GetImageFromArray(outovr.astype(np.short))
                itk.imwrite(itkimg, new_name, compression=True)

In [5]:
output_pathname = "../../Data/VFoldData/ROIData"

#pathname = "../../Data/VFoldData/BAMC-PTXNoSliding"
pathname = "../../Data/VFoldData/BAMC-PTXSliding"

extract_columns(pathname, output_pathname, 16)

32 32
(61, 320, 320)
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/004s_iimage_73815992352100_clean.extruded-overlay.mha
75 288
13 220
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/004s_iimage_74132233134844_clean.extruded-overlay.mha
30 319
17 124
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10705997566592_CLEAN.extruded-overlay.mha
122 212
99 290
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10891015221417_clean.extruded-overlay.mha
110 253
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/030s_iimage_1180496934444_clean.extruded-overlay.mha
68 270
102 319
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/030s_iimage_677741729740_clean.extruded-overlay.mha
80 275
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/034s_iimage_3368391807672_clean.extruded-overlay.mha
140 256
116 242
99 222
Processing ../../Data/VFoldData/BAMC-PTX